In [1]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import re
import pickle
import nltk
import string
import collections

In [3]:
# config to access tweeter API
config = configparser.ConfigParser()
config.read('config.ini')

consumer_key = config['twitter']['consumer_key']
consumer_secret = config['twitter']['consumer_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [4]:
# authentication
client = tweepy.Client(consumer_key=consumer_key, consumer_secret=consumer_secret,
                                   access_token=access_token, access_token_secret=access_token_secret,wait_on_rate_limit=True)

In [5]:
# get_tweets only return 100 results, handle the case when there is more than 100
def lookup_tweets(tweet_IDs, client):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    for i in range(int((tweet_count / 100) + 1)):
        # Catch the last group if it is less than 100 tweets
        end_loc = min((i + 1) * 100, tweet_count)
        if tweet_IDs[i * 100:end_loc]:
            tweets = client.get_tweets(tweet_IDs[i * 100:end_loc],user_auth=True).data
            if tweets:
                full_tweets.extend(tweets)
    return full_tweets
    

In [13]:
# read data 
covid_data_file = open('project-data/covid.data.txt', 'r')
covid_lines = covid_data_file.readlines()
covid_events =[]
# Strips the newline character
for line in covid_lines:
    covid_events.append(list(map(int,line.strip('\n').split(','))))

In [ ]:
# get the text of all events
covid_events_text=[]
for event in covid_events:
    results = lookup_tweets(event, client)
    covid_event_text=[tweet.text for tweet in results]
    covid_events_text.append(covid_event_text)
    

In [ ]:
# save data to pickle file
f = open(f'./covid_text.pckl','wb')
pickle.dump(covid_data,f)
f.close()

In [ ]:
# Preprocessing covid data

f = open(f'./covid_text.pckl','rb')
covid_data = pickle.load(f)
f.close()


In [ ]:
## cleaning the tweets
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #remove @mention
    text = re.sub(r'#','',text) # remove the hashtag symbol
    text = re.sub(r'https?:\/\/\S+', '',text) #remove hyperlink
    text = re.sub(r'\n','',text) # remove \n 
    text = re.sub(r'\r','',text) # remove \r
    text = re.sub(r'[0-9]+','',text) #remove all the number
    text = re.sub(r'\W+', ' ', text) #remove special characters
    text = text.strip().lower()
    if len(text) != 0:
        return text
    else:
        return None
    
for i in range(len(covid_data)):
    for j in range(len(covid_data[i])):
        covid_data[i][j] = clean_text(covid_data[i][j])
    covid_data[i] = [x for x in covid_data[i] if x is not None]
        

In [ ]:
# merge source tweeet and reply tweet together for covid data    
covid_merge_events=[]
for event in covid_data:
    merge = ''
    for tweet in event:
        merge = merge + tweet
    covid_merge_events.append(merge)

In [ ]:
# drop all null value and export to csv file
covid_df = pd.DataFrame({'text':covid_merge_events})

nan_value = float("NaN")

covid_df.replace("", nan_value, inplace=True)

covid_df.dropna(axis=0 ,inplace=True)

covid_df.to_csv('covid.tsv', sep='\t',index=False)

In [ ]:
!pip install torch torchvision transformers

In [ ]:
from transformers import BertModel
from transformers import BertTokenizer


#load BERT base uncased model
bert_model = BertModel.from_pretrained('bert-base-uncased')

#load BERT's WordPiece tokenisation model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class TweetClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits,outputs

In [ ]:
gpu = 0 #gpu ID

print("Creating the tweet classifier, initialised with pretrained BERT-BASE parameters...")
net = TweetClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the tweet classifier.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
weight_file = "/content/drive/MyDrive/NLP/bert_merge.dat"

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class Dataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [ ]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = Dataset(filename = 'train_bert_merge.tsv', maxlen = 512)
dev_set = Dataset(filename = 'dev_bert_merge.tsv', maxlen = 512)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set,batch_size = 8, num_workers = 0)
dev_loader = DataLoader(dev_set, batch_size = 8, num_workers = 0)

print("Done preprocessing training and development data.")

In [ ]:
# get embedded layer for train set
train_outputs= None
net.load_state_dict(torch.load(weight_file))
with torch.no_grad():
    for seq, attn_masks,_ in train_loader:
        seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
        logits,output = net(seq, attn_masks)
        pooled_output = output.pooler_output   
        if train_outputs is None:
            train_outputs = pooled_output.detach().cpu().numpy()
        else:
            train_outputs = np.append(train_outputs, pooled_output.detach().cpu().numpy(), axis=0)

# get embedded layer for dev set
dev_outputs= None
net.load_state_dict(torch.load(weight_file))
with torch.no_grad():
    for seq, attn_masks,_ in dev_loader:
        seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
        logits,output = net(seq, attn_masks)
        pooled_output = output.pooler_output   
        if dev_outputs is None:
            dev_outputs = pooled_output.detach().cpu().numpy()
        else:
            dev_outputs = np.append(dev_outputs, pooled_output.detach().cpu().numpy(), axis=0)

In [ ]:
# get label for train and dev set
train = pd.read_csv('train_bert_merge.tsv', sep='\t')
dev = pd.read_csv('dev_bert_merge.tsv', sep='\t')
train_labels = train['label']
dev_labels = dev['label']

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd

class CovidDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask

In [ ]:
covid_set = CovidDataset(filename = 'covid.tsv', maxlen = 512)

#Creating intsances of training and development dataloaders
covid_loader = DataLoader(covid_set,batch_size = 1, num_workers = 0)

# get embedded layer for covid set
covid_outputs= None
net.load_state_dict(torch.load(weight_file))
with torch.no_grad():
    for seq, attn_masks in covid_loader:
        seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
        logits,output = net(seq, attn_masks)
        pooled_output = output.pooler_output   
        if covid_outputs is None:
            covid_outputs = pooled_output.detach().cpu().numpy()
        else:
            covid_outputs = np.append(covid_outputs, pooled_output.detach().cpu().numpy(), axis=0)

In [ ]:
# Apply Linear SVC for embedded layer

from sklearn.svm import LinearSVC
SVC = LinearSVC(max_iter=10000)
SVC.fit(train_outputs,np.array(train_labels))

svc_predict = SVC.predict(covid_outputs)    
df = pd.DataFrame({"text": covid_df['text'] ,"Predicted": svc_predict}) 
df.to_csv('covid_predict.csv',index=False)